# Setting

In [ ]:
import os
import glob
import math
import pandas as pd

import random

import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm
import random as r
import wandb
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Seed the behavior of the environment variable
os.environ['PYTHONHASHSEED'] = str(1)
# Seed numpy's instance in case you are using numpy's random number generator, shuffling operations, ...
np.random.seed(1)
# Seed Python's random number generator, in case you are using Python's random number generator, shuffling operations, ...
r.seed(1)



In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
set_seed

In [ ]:
BASE_MASK_PATH = '/opt/ml/input/data/'
TRAIN_MASK_IMAGE_PATH = os.path.join(BASE_MASK_PATH, 'train/images/')
TRAIN_MASK_LABEL_PATH = os.path.join(BASE_MASK_PATH, 'train/train.csv')
TEST_MASK_IMAGE_PATH = os.path.join(BASE_MASK_PATH, 'test/images/')
TEST_MASK_LABEL_PATH = os.path.join(BASE_MASK_PATH, 'test/info.csv')

In [ ]:
train_data = pd.read_csv('/opt/ml/input/data/train/train.csv')
train_pre_data = pd.read_csv('/opt/ml/input/data/train/train_pre.csv')
train_pre_data['gender'] = train_pre_data['gender'].map({'male':0, 'female':1})
train_pre_data.head()

# Data

## Dataset

In [ ]:
img_path = train_pre_data['path']
labels = train_pre_data['class']

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, img_paths, labels, transform):
        self.img_paths = TRAIN_MASK_IMAGE_PATH + img_paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        x = Image.open(self.img_paths[idx])
        y = self.labels[idx]
  
        if self.transform:
            x = self.transform(image=np.array(x))['image']
        return x, y
        
    def __len__(self):
        return len(self.img_paths)

In [ ]:
# train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(img_path, labels, test_size=0.2, stratify=labels, shuffle=True, random_state=30)
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)

In [ ]:
from albumentations import *
from albumentations.pytorch import ToTensorV2
def get_transforms(need=('train', 'val'), img_size=(384, 384), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):

    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            # Resize(img_size[0], img_size[1], p=1.0),
            CenterCrop(img_size[0], img_size[1], p=1.0),
            HorizontalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            GaussNoise(p=0.5),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            # Resize(img_size[0], img_size[1]),
            CenterCrop(img_size[0], img_size[1], p=1.0),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

In [ ]:
transform = get_transforms()

mask_train_dataset = TrainDataset(x_train, y_train, transform['train'])
mask_valid_dataset = TrainDataset(x_valid, y_valid, transform['val'])

## DataLoader

In [ ]:
from multiprocessing import cpu_count
# num_workers = int(cpu_count() / 2)
num_workers = 1
batch_size = 32

mask_train_dataloader = DataLoader(
    mask_train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
    )

mask_valid_dataloader = DataLoader(
    mask_valid_dataset,
    batch_size=batch_size,
    )


In [ ]:
images, labels = next(iter(mask_train_dataloader))
labels

# Modeling

In [ ]:
efficient = EfficientNet.from_pretrained('efficientnet-b4')
for param in efficient.parameters():
    param.requires_grad = False
efficient._fc = torch.nn.Linear(in_features=1792, out_features=18, bias=True)

# torch.nn.init.xavier_uniform_(efficient._fc.weight)
# stdv = 1. / math.sqrt(efficient._fc.weight.size(1))
# efficient._fc.bias.data.uniform_(-stdv, stdv)

model = efficient

# Train

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

model.to(device)

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 20 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션. 여러 값을 사용해보세요1

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train" : mask_train_dataloader,
    "valid" : mask_valid_dataloader
}

In [ ]:
import wandb
wandb.init(project='mask_contest', entity='sala0320', config={"epochs" : NUM_EPOCH, "batch_size": batch_size, "lr" : LEARNING_RATE})
wandb.run.name = "Basic Test Efficientnet-b4"

os.environ["WANDB_API_KEY"] = '88ec5255a3b1e91fb1eb92834a31072727d5ada2'
os.environ["WANDB_MODE"] = "dryrun"

In [ ]:
### 학습 코드 시작
set_seed(42)
best_test_accuracy = 0.
best_test_loss = 100.
early_stopping = 0.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "valid"]:
    running_loss = 0.
    running_acc = 0.
    gt = []
    pred = []

    if phase == "train":
      model.train() 
     
    elif phase == "valid":
      model.eval() 
  
    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):

      images = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad() 

      with torch.set_grad_enabled(phase == "train"): 
        logits = model(images)
        _, preds = torch.max(logits, 1) 
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward()
          optimizer.step() 
          # scheduler.step() 

      running_loss += loss.item() * images.size(0) 
      running_acc += torch.sum(preds == labels.data) 
      gt.extend(labels.tolist())
      pred.extend(preds.tolist())

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)
    epoch_f1 =  f1_score(gt, pred, average='micro')
    wandb.log({"epoch_loss":  epoch_loss,
               "epoch_acc":  epoch_acc,
               "epoch_f1" : epoch_f1})

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}, F1 : {epoch_f1:.3f}")

    if phase == "valid" and best_test_accuracy < epoch_acc:
      best_test_accuracy = epoch_acc

    if phase == "valid" and best_test_loss > epoch_loss:
      best_test_loss = epoch_loss
      torch.save(model.state_dict(), '/opt/ml/input/data/model/best_model.pth')
      early_stopping = 0.

    elif phase == "valid" and best_test_loss < epoch_loss:
      early_stopping += 1
      print("Early stopping! " + str(early_stopping))
    
    if early_stopping > 3:
      break

    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")


# Testing

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b4')
model._fc = torch.nn.Linear(in_features=1792, out_features=18, bias=True)
model.load_state_dict(torch.load('/opt/ml/input/data/train/best_model.pth'))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = TRAIN_MASK_IMAGE_PATH + img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = train_pre_data['path']
transform = transforms.Compose([
    transforms.CenterCrop(380),
    # Resize((380, 380), Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
# model = MyModel(num_classes=18).to(device)
model = model.to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())

In [ ]:
test_pre_data = train_pre_data
test_pre_data['predict'] = all_predictions
for i in range(len(test_pre_data)):
    if test_pre_data['class'][i] != test_pre_data['predict'][i]:
        print(f"path : {test_pre_data['path'][i]} real : {test_pre_data['class'][i]} predict : {test_pre_data['predict'][i]}")

In [ ]:
import matplotlib.pyplot as plt
imgs = submission['ImageID']
label = submission['ans']
fig, axes = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(12, 6))
for i, j in enumerate(range(5)):
    axes[i].imshow(Image.open('/opt/ml/input/data/eval/images/' + imgs[j]))
    axes[i].set_title(f'{label[j]}')
plt.show()